In [ ]:
from pathlib import Path
from datetime import datetime, timedelta

import polars as pl

import stock

In [ ]:
csv_path = stock.PROJECT_ROOT / "data" / "watch_list_jp.csv"
watch_list = pl.read_csv(csv_path)["code"].to_numpy()

In [ ]:
nikkei_df = stock.kabutan.read_data_csv(stock.PROJECT_ROOT / "data" / "daily" / "0000.csv", exclude_none=False)

def get_start_and_high(df: pl.DataFrame):
    start = df["open"][0]
    high = df["close"].mean()
    return start, high

In [ ]:
def calc_relative_strength(df: pl.DataFrame, ref_df: pl.DataFrame, start: datetime, end: datetime):
    df = df.filter((start < pl.col("date") & (pl.col("date") <= end)))
    ref_df = ref_df.filter((start < pl.col("date") & (pl.col("date") <= end)))

    if len(df) != len(ref_df):
        return -1
    
    return stock.relative_strength.relative_strength(
        df["close"].to_numpy(), ref_df["close"].to_numpy()
    )

In [ ]:
durations = []
for code in watch_list:
    start_date = datetime.today() - timedelta(days=365)
    durations.append(stock.trend_template.calc_watch_list_duration_of(code, start_date))
    print(code, durations[-1][0][0].strftime("%Y/%m/%d"), durations[-1][-1][1].strftime("%Y/%m/%d"))

In [ ]:
cnt = 0
for idx, code in enumerate(watch_list):
    duration = durations[idx]
    data_csv_path = stock.PROJECT_ROOT / "data" / "daily" / f"{code}.csv"
    df = stock.kabutan.read_data_csv(data_csv_path, exclude_none=True)
    filtered_df = df.filter(pl.col("date").is_between(duration[0][0], duration[-1][1], closed="right"))
    if len(filtered_df) == 0:
        continue
    start, high = get_start_and_high(filtered_df)

    filtered_nikkei_df = nikkei_df.filter(pl.col("date").is_between(duration[0][0], duration[-1][1], closed="right"))
    n_start, n_high = get_start_and_high(filtered_nikkei_df)

    print(code, high / start, n_high / n_start)
    if high / start  > n_high / n_start:
        cnt += 1
